In [35]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline

In [2]:
df_rando = pd.read_csv("RandoGironde.csv")

In [3]:
df_rando.drop(columns = "Unnamed: 0", inplace = True)

In [4]:
display(df_rando.info(),
        df_rando.describe(),
        df_rando.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        200 non-null    object
 1   Town        200 non-null    object
 2   Types       199 non-null    object
 3   Distance    200 non-null    object
 4   Time        199 non-null    object
 5   Difficulty  199 non-null    object
 6   Image       200 non-null    object
dtypes: object(7)
memory usage: 11.1+ KB


None

,Name,Town,Types,Distance,Time,Difficulty,Image
count,200,200,199,200,199,199,200
unique,152,113,2,87,50,4,149
top,Boucle pédestre : Les Artigues de Lussac en t...,arcachon,à pied,10 km,Durée : 2h,Difficulté : Facile,https://www.gironde-tourisme.com/wp-content/up...
freq,2,11,198,10,27,105,3


,Name,Town,Types,Distance,Time,Difficulty,Image
0,Saint-Laurent-du-Plan grande boucle,saint-laurent-du-plan,à pied,8 km,Durée : 1h30,Difficulté : Facile,https://www.gironde-tourisme.com/wp-content/up...
96,Le sentier du Lac à Maubuisson,carcans-maubuisson,à pied,7 km,Durée : 2h,Difficulté : Facile,https://www.gironde-tourisme.com/wp-content/up...
146,Boucle pédestre du Lac de Saint-Christoly-de-...,saint-christoly-de-blaye,à pied,"9,66 km",Durée : 2h,Difficulté : Très facile,https://www.gironde-tourisme.com/wp-content/up...
155,Boucle du Bois Bourru à Pellegrue,pellegrue,à pied,"15,4 km",Durée : 4h,Difficulté : Moyenne,https://www.gironde-tourisme.com/wp-content/up...
148,"Au Teich, comme un oiseau sur le sentier du l...",le teich,à pied,8 km,Durée : 2h30,Difficulté : Facile,https://www.gironde-tourisme.com/wp-content/up...


# Fill NA

In [5]:
# Fill NA
df_rando[df_rando["Types"].isna()]

,Name,Town,Types,Distance,Time,Difficulty,Image
81,Route pittoresque de la Corniche,gauriac,NaN,10 km,NaN,NaN,https://www.gironde-tourisme.com/wp-content/up...


In [6]:
# Données récupéré depuis google
df_rando["Time"][df_rando["Types"].isna()] = "Durée : 1h00"
df_rando["Difficulty"][df_rando["Types"].isna()] = "Difficulté : Facile"
df_rando["Types"][df_rando["Types"].isna()] = "à vélo"

C:\Users\User\AppData\Local\Temp\ipykernel_19908\645246321.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_rando["Time"][df_rando["Types"].isna()] = "Durée : 1h00"
C:\Users\User\AppData\Local\Temp\ipykernel_19908\645246321.py:3: Futur

# Numerize Distance

In [7]:
df_rando["Distance"] =  df_rando["Distance"].apply(lambda x: float(x.replace(" km", "").replace(",", ".")))

# Transform Time in Duration in minutes

In [8]:
df_rando["Time"].unique()

array([' Durée : 1h30  ', ' Durée : 2h  ', ' Durée : 1h15  ',
       ' Durée : 2  ', ' Durée : 1h45  ', ' Durée : 3h  ', ' Durée : 3  ',
       ' Durée : 6h15  ', ' Durée : 2h10  ', ' Durée : 3H30  ',
       ' Durée : 1h  ', ' Durée : 7h  ', ' Durée : 4  ',
       ' Durée : 4h00  ', ' Durée : 4h  ', ' Durée : 2 Jours  ',
       ' Durée : 1h25  ', ' Durée : 45 mn  ', ' Durée : 1h10  ',
       ' Durée : 2h45  ', ' Durée : 5h  ', ' Durée : 30 mn  ',
       ' Durée : 2h20  ', ' Durée : 2h30  ', ' Durée : 3h30  ',
       ' Durée : 3h15  ', ' Durée : 45 min  ', ' Durée : 3h00  ',
       ' Durée : 30mn  ', ' Durée : 4h15  ', ' Durée : 3h45  ',
       ' Durée : 4h30  ', ' Durée : 1 jour  ', ' Durée : 7h30  ',
       'Durée : 1h00', ' Durée : 1h00  ', ' Durée : 2h15  ',
       ' Durée : 1h20  ', ' Durée : 3,30  ', ' Durée : 1h50  ',
       ' Durée : 3h45/4h  ', ' Durée : 1H30  ', ' Durée : 4 h  ',
       ' Durée : 3h10  ', ' Durée : 45mn  ', ' Durée : 6h30  ',
       ' Durée : 6h10  ', ' Durée 

In [9]:
# Virer duree et mettre au même format
df_rando["Time"] = df_rando["Time"].apply(lambda x: x.replace("Durée : ", "").replace("mn", "min").replace("H", "h").replace("  ", "").replace(" ", "").replace("00", "").replace(",", "h").replace("3h45/4h", "4h").replace("1jour", "24h").replace("2Jours", "48h"))

df_rando["Time"].unique()

array(['1h30', '2h', '1h15', '2', '1h45', '3h', '3', '6h15', '2h10',
       '3h30', '1h', '7h', '4', '4h', '48h', '1h25', '45min', '1h10',
       '2h45', '5h', '30min', '2h20', '2h30', '3h15', '4h15', '3h45',
       '4h30', '24h', '7h30', '2h15', '1h20', '1h50', '3h10', '6h30',
       '6h10', '6h', '1h40'], dtype=object)

In [10]:
df_rando[(df_rando["Time"] == "3") | (df_rando["Time"] == "4") | (df_rando["Time"] == "2")]

,Name,Town,Types,Distance,Time,Difficulty,Image
4,De Sainte-Foy-la-Grande à Port-Sainte-Foy-et-...,sainte-foy-la-grande,à pied,8.0,2,Difficulté : Facile,https://www.gironde-tourisme.com/wp-content/up...
8,Boucle Haut Val d’Engranne Sud,gornac,à pied,10.2,3,Difficulté : Facile,https://www.gironde-tourisme.com/wp-content/up...
21,La Sauve-Majeure grande boucle,la sauve,à pied,15.0,4,Difficulté : Facile,https://www.gironde-tourisme.com/wp-content/up...
113,La Sauve-Majeure grande boucle,la sauve,à pied,15.0,4,Difficulté : Facile,https://www.gironde-tourisme.com/wp-content/up...


In [11]:
# Correction des temps incohérents
df_rando["Time"].iloc[4] = "2h"
df_rando["Time"].iloc[8] = "3h"
df_rando["Time"].iloc[21] = "4h"
df_rando["Time"].iloc[113] = "4h"

C:\Users\User\AppData\Local\Temp\ipykernel_19908\3758786976.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_rando["Time"].iloc[4] = "2h"
C:\Users\User\AppData\Local\Temp\ipykernel_19908\3758786976.py:2: SettingWithCopyWarning: 
A valu

In [12]:
df_rando["Time"].unique()

array(['1h30', '2h', '1h15', '1h45', '3h', '6h15', '2h10', '3h30', '1h',
       '7h', '4h', '48h', '1h25', '45min', '1h10', '2h45', '5h', '30min',
       '2h20', '2h30', '3h15', '4h15', '3h45', '4h30', '24h', '7h30',
       '2h15', '1h20', '1h50', '3h10', '6h30', '6h10', '6h', '1h40'],
      dtype=object)

In [13]:
df_rando["Time"].value_counts()

Time
2h       28
3h       25
1h30     19
2h30     17
1h       16
4h       13
3h30     11
1h45      9
45min     7
30min     6
1h15      6
5h        5
7h        3
3h15      3
1h50      3
1h10      3
2h45      3
7h30      2
24h       2
4h30      2
3h45      2
6h15      2
2h10      2
4h15      1
48h       1
2h20      1
2h15      1
1h20      1
1h25      1
3h10      1
6h30      1
6h10      1
6h        1
1h40      1
Name: count, dtype: int64

In [14]:
"1h10" < "1h30"

True

In [15]:
def dur(time):
    
    Duration = [{"1h": "<= 1h"}, {"2h": "<= 2h"}, {"3h": "<= 3h"}, {"5h": "<= 5h"}, {"8h": "<= 8h"}, {"48h": "> 8h"}]
    
    if time == "30min" or time == "45min":
        return "<= 1h"
    else:
        for ele in range(len(Duration)):
            if time <= [key for key in Duration[ele]][0]:
                return [val for val in Duration[ele].values()][0]

In [16]:
df_rando["Duration"] = df_rando["Time"].apply(lambda x: dur(x))

In [17]:
df_rando["Duration"].value_counts()

Duration
<= 2h    73
<= 3h    49
<= 5h    39
<= 1h    29
<= 8h    10
Name: count, dtype: int64

In [18]:
df_rando[(df_rando["Time"] == "24h") | (df_rando["Time"] == "48h")]

,Name,Town,Types,Distance,Time,Difficulty,Image,Duration
25,Chemin de Saint-Jacques-Compostelle par la Ch...,le verdon-sur-mer,à pied,96.0,48h,Difficulté : Moyenne,https://www.gironde-tourisme.com/wp-content/up...,<= 5h
74,Urban Wine Trail,bordeaux,à pied,25.0,24h,Difficulté : Facile,https://www.gironde-tourisme.com/wp-content/up...,<= 2h
193,Urban Wine Trail,bordeaux,à pied,25.0,24h,Difficulté : Facile,https://www.gironde-tourisme.com/wp-content/up...,<= 2h


In [19]:
df_rando['Duration'].iloc[25] = "> 8h"
df_rando['Duration'].iloc[74] = "> 8h"
df_rando['Duration'].iloc[193] = "> 8h"

C:\Users\User\AppData\Local\Temp\ipykernel_19908\973722224.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_rando['Duration'].iloc[25] = "> 8h"
C:\Users\User\AppData\Local\Temp\ipykernel_19908\973722224.py:1: SettingWithCopyWarning: 
A

In [20]:
df_rando["Difficulty"] = df_rando["Difficulty"].apply(lambda x: x.replace(" Difficulté : ", "").replace("Difficulté : ", ""))

In [30]:
df_rando["Difficulty"].value_counts()

Difficulty
Facile          106
Moyenne          52
Très facile      35
Difficile         7
Name: count, dtype: int64

In [29]:
df_rando["Difficulty"] = df_rando["Difficulty"].apply(lambda x: x.replace("Moyenne ", "Moyenne").replace("Très Facile ", "Très Facile").replace("Facile ", "Facile").replace("Difficile ", "Difficile"))

In [32]:
display(df_rando.info(),
        df_rando.describe(),
        df_rando.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Name        200 non-null    object 
 1   Town        200 non-null    object 
 2   Types       200 non-null    object 
 3   Distance    200 non-null    float64
 4   Time        200 non-null    object 
 5   Difficulty  200 non-null    object 
 6   Image       200 non-null    object 
 7   Duration    200 non-null    object 
 8   Range       200 non-null    object 
dtypes: float64(1), object(8)
memory usage: 14.2+ KB


None

,Distance
count,200.000000
mean,10.103800
std,8.573147
min,0.600000
25%,5.675000
50%,9.000000
75%,12.300000
max,96.000000


,Name,Town,Types,Distance,Time,Difficulty,Image,Duration,Range
95,Les secrets de la petite Amazone,le teich,à pied,10.0,3h,Très facile,https://www.gironde-tourisme.com/wp-content/up...,<= 3h,<= 10km
12,Chemin d’Amadour : étape 1 de Soulac-sur-Mer ...,soulac-sur-mer,à pied,25.5,6h15,Moyenne,https://www.gironde-tourisme.com/wp-content/up...,<= 8h,> 15km
65,Boucle pédestre : A la croisée des appellations,saint-christophe-des-bardes,à pied,11.1,2h30,Moyenne,https://www.gironde-tourisme.com/wp-content/up...,<= 3h,<= 15km
175,Boucle de la Bassanne,savignac,à pied,11.0,2h45,Facile,https://www.gironde-tourisme.com/wp-content/up...,<= 3h,<= 15km
18,Village de Verdelais,verdelais,à pied,3.5,1h,Facile,https://www.gironde-tourisme.com/wp-content/up...,<= 1h,<= 6km


In [23]:
def rangeD(distance):
    if distance <= 6:
        return "<= 6km"
    elif distance <= 10:
        return "<= 10km"
    elif distance <= 15:
        return "<= 15km"
    else:
        return "> 15km"
    
df_rando["Range"] = df_rando["Distance"].apply(lambda x: rangeD(x))


In [40]:
X = df_rando[["Types", "Range", "Difficulty", "Duration"]]

Rando_folle = make_pipeline(OneHotEncoder(),
                            NearestNeighbors(n_neighbors = 3))
 
Rando_folle = Rando_folle.fit(X)

In [38]:
UserPref = pd.DataFrame({"Types": ["à pied"],
                         "Range": ["<= 15km"],
                         "Difficulty": ["Moyenne"],
                         "Duration": ["<= 8h"]})
UserPref

,Types,Range,Difficulty,Duration
0,à pied,<= 15km,Moyenne,<= 8h


In [41]:
closestR = Rando_folle.kneighbors(UserPref, X.columns)


AttributeError: 'Pipeline' object has no attribute 'kneighbors'